# Music Popularity Prediciton
DJ Khaled boldly claimed to always know when a song will be a hit. We decided to further investigate by asking three key questions: 
1. Are there certain characteristics for hit songs 
2. what are the largest influencers on a song’s success 
3. and can old songs even predict the popularity of new songs? 

## Creating Dataset
There are no relevant and updated dataset that exists to perform this analysis, hence we decided to form our our dataset by using a technique called Data Collection.
For this project, we'll be using the data produced from one of the largest Music Streaming Website in the world called Spotify. We'll be using Spotify Web API to collect our raw data from the custom playlist of 1300 songs. 
But for demonstrative purpose we will be using a smaller playlist with 32 songs for now.

## DataFrames
Our dataset will contain 3 data frames namely, Album Data, Artist Data, Song Data.
In the following code I have demonstrated the creation of the Song DataFrame.

In [1]:
#Import Library
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import spotipy
from matplotlib import style
from spotipy import util
from spotipy.oauth2 import SpotifyClientCredentials 
loc='G:\\Projects\\Music-Popularity-Prediction\\DataMining'

### Step 1 : Setting Up Connection
Set Up Connection with Spotify Web API and give it access to my spotify account using a token generated automatically. We need this to use my 32 song playlist called 'Best Mixtape Ever'

In [2]:
#Set up Connection 
client_id = 'a90f2a20aea54d5781d4cc7a38ab6694' #Need to create developer profile
client_secret = 'b8810bc6ac1d4877b78c45a8654ebf18'
username = 'f0uthgq19mkamht0gno58wkt0' #Store username
scope = 'playlist-read-collaborative'
redirect_uri='http://localhost/'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
client_secret=client_secret)#Create manager for ease
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, client_id, 
client_secret, redirect_uri)

if token:
 sp = spotipy.Spotify(auth=token)
else:
 print("Can't get token for", username)

In [3]:
#Connect to the 'the ultimate playlist'
playlist_id = 'spotify:playlist:0vW19IAl4yjeg2IlfNuBFh'
playlist = sp.user_playlist(username,playlist_id)

### Step 2 : Collecting the Data

In [4]:
def user_playlist_tracks_full(spotify_connection, user, playlist_id=None, fields=None, market=None):
    """ Get full details of the tracks of a playlist owned by a user.
        https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/

        Parameters:
            - user - the id of the user
            - playlist_id - the id of the playlist
            - fields - which fields to return
            - market - an ISO 3166-1 alpha-2 country code.
    """

    # first run through also retrieves total no of songs in library
    response = spotify_connection.user_playlist_tracks(user, playlist_id, fields=fields, limit=100, market=market)
    results = response["items"]
    

    # subsequently runs until it hits the user-defined limit or has read all songs in the library
    while len(results) < response["total"]:
        response = spotify_connection.user_playlist_tracks(user, playlist_id, fields=fields, limit=100, offset=len(results), market=market)
        results.extend(response["items"]) 
    return(results)


In [5]:
results=user_playlist_tracks_full(sp,username,playlist_id)

In [6]:
raw_playlist_data=[]
for i in range(0,len(results)):
    if results[i]['track']['id'] != None: # Removes  local tracks, if any
        play=results[i]
        raw_playlist_data.append(play['track'])

# Artist Table

In [7]:
artistData=[]
for i in range(0,len(raw_playlist_data)):
    artistData.append(raw_playlist_data[i]['artists'][0])

In [10]:
#correct the format of external link, delete unwanted data and append to artistId
for i in range(len(artistData)):
    artistData[i]['url']=artistData[i]['external_urls']['spotify']
    #artistData[i]['artist id'] = artistData[i]['id']
    del artistData[i]['href']
    del artistData[i]['external_urls']
    del artistData[i]['type']
    #del artistData[i]['id']
    

In [11]:
artistId=[]
for i in range(len(artistData)):
    artistId.append(artistData[i]['id'])

In [12]:
artistData

[{'id': '5YGY8feqx7naU7z4HrwZM6',
  'name': 'Miley Cyrus',
  'uri': 'spotify:artist:5YGY8feqx7naU7z4HrwZM6',
  'url': 'https://open.spotify.com/artist/5YGY8feqx7naU7z4HrwZM6'},
 {'id': '7n2wHs1TKAczGzO7Dd2rGr',
  'name': 'Shawn Mendes',
  'uri': 'spotify:artist:7n2wHs1TKAczGzO7Dd2rGr',
  'url': 'https://open.spotify.com/artist/7n2wHs1TKAczGzO7Dd2rGr'},
 {'id': '246dkjvS1zLTtiykXe5h60',
  'name': 'Post Malone',
  'uri': 'spotify:artist:246dkjvS1zLTtiykXe5h60',
  'url': 'https://open.spotify.com/artist/246dkjvS1zLTtiykXe5h60'},
 {'id': '4GNC7GD6oZMSxPGyXy4MNB',
  'name': 'Lewis Capaldi',
  'uri': 'spotify:artist:4GNC7GD6oZMSxPGyXy4MNB',
  'url': 'https://open.spotify.com/artist/4GNC7GD6oZMSxPGyXy4MNB'},
 {'id': '246dkjvS1zLTtiykXe5h60',
  'name': 'Post Malone',
  'uri': 'spotify:artist:246dkjvS1zLTtiykXe5h60',
  'url': 'https://open.spotify.com/artist/246dkjvS1zLTtiykXe5h60'},
 {'id': '1LnBqMojv6Dv6DoPWTDSZ7',
  'name': 'Saco',
  'uri': 'spotify:artist:1LnBqMojv6Dv6DoPWTDSZ7',
  'url': '

In [15]:
genres=[]
for i in range(len(artistId)):
    response=""
    response=sp.artist(artistId[i])
    genres.append(response['genres'])
    artistData[i]['genres']=response['genres']
    artistData[i]['artist popularity']=response['popularity']

SSLError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/artists/1HBjj22wzbscIZ9sEb5dyf (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')")))

In [12]:
#list of all unique genres in my dataset
l=[]
for i in genres:
    for j in i:
        if j not in l:
            l.append(j)

In [13]:
d={}
n=0
for i in l:
    d[i]=n
    n+=1
d    

{'dance pop': 0,
 'pop': 1,
 'post-teen pop': 2,
 'canadian pop': 3,
 'viral pop': 4,
 'dfw rap': 5,
 'melodic rap': 6,
 'rap': 7,
 'uk pop': 8,
 'tropical house': 9,
 'hip hop': 10,
 'new orleans rap': 11,
 'pop rap': 12,
 'trap': 13,
 'deep tropical house': 14,
 'arab electronic': 15,
 'edm': 16,
 'electropop': 17,
 'metropopolis': 18,
 'electro house': 19,
 'house': 20,
 'progressive house': 21,
 'uk dance': 22,
 'pop house': 23,
 'dancehall': 24,
 'australian dance': 25,
 'australian pop': 26,
 'country rap': 27,
 'lgbtq+ hip hop': 28,
 'girl group': 29,
 'talent show': 30,
 'canadian contemporary r&b': 31,
 'indie poptimism': 32,
 'complextro': 33,
 'german techno': 34,
 'folk-pop': 35,
 'modern alternative rock': 36,
 'modern rock': 37,
 'stomp pop': 38,
 'neo mellow': 39,
 'pop rock': 40,
 'aussietronica': 41,
 'contemporary country': 42,
 'country pop': 43,
 'latin': 44,
 'latin pop': 45,
 'mexican pop': 46,
 'tropical': 47,
 'moombahton': 48,
 'miami hip hop': 49,
 'latin hip 

In [14]:
artistData[0]['genres'][0]

'dance pop'

In [15]:
artistData[0]

{'id': '5YGY8feqx7naU7z4HrwZM6',
 'name': 'Miley Cyrus',
 'uri': 'spotify:artist:5YGY8feqx7naU7z4HrwZM6',
 'url': 'https://open.spotify.com/artist/5YGY8feqx7naU7z4HrwZM6',
 'genres': ['dance pop', 'pop', 'post-teen pop'],
 'artist popularity': 87}

In [16]:
L=[]
for i in range(len(artistData)):
    zeros=[0]*257
    D={}
    for j in artistData[i]['genres']:
        zeros[d[j]]=1
    D[artistId[i]]=zeros
    L.append(D)
#[artistId[i]]=

In [20]:
L[1]

{'7n2wHs1TKAczGzO7Dd2rGr': [0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [39]:
genreDf=pd.DataFrame(L)
genreDf

,00FQb4jTyendYWaN8pK0wa,02A3cEvlLLCbIMVDrK2GHV,02kJSzxNuaWGqwubyUba0Z,04gDigrS5kc9YWfZHwBETP,06HL4z0CvFAxyc27GXpf02,07Q9n9pbwR6jQ8xH4SA46b,07QEuhtrNmmZ0zEcqE9SF6,07YZf4WDAMNwqr4jfgOZ8y,085pc2PYOi8bGKj0PNjekA,0988g1pgIlWAKBVf3DQ7db,...,7pGyQZx9thVa8GxMBeXscB,7qG3b048QCHVRO5Pv1T5lw,7rZNSLWMjTbwdLNskFbzFf,7sfgqEdoeBTjd8lQsPT3Cy,7sfl4Xt5KmfyDs2T3SVSMK,7vXwfZyDp3spzIVNXDaTPN,7vk5e3vY1uw9plTHJAMwjN,7x8nK0m0cP2ksQf0mjWdPS,7xTcuBOIAAIGDOSvwYFPzk,7yKYcY9cyMNgynj4QzGrCq
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
for i, row in genreDf.iterrows():
      genreDf.at[i, "dance pop"] = D[i[0][0]]
genreDf
        

KeyError: '5'

In [113]:
df1=pd.DataFrame([artistData][0])

In [114]:
df1.rename(columns = {'id':'artist_id'}, inplace=True)

In [115]:
final_df = df.drop_duplicates('artist id').sort_index()
print("in the list of " + str(len(df))+ " songs, there are only "+str(len(final_df))+" unique artists")

in the list of 1491 songs, there are only 430 unique artists


In [41]:
final_df.to_csv(loc+"\\ArtistData.csv")

# Album Data

In [55]:
albumData=[]
for i in range(len(raw_playlist_data)):
    albumData.append(raw_playlist_data[i]['album'])

In [56]:
for i in range(len(albumData)):
    albumData[i]['url']=albumData[i]['external_urls']['spotify']
    albumData[i]['image']=albumData[i]['images'][0]['url']
    del albumData[i]['artists']
    del albumData[i]['available_markets']
    del albumData[i]['external_urls']
    del albumData[i]['href']
    del albumData[i]['images']
    del albumData[i]['release_date_precision']
    del albumData[i]['type']

KeyError: 'external_urls'

In [57]:
albumData[0]

{'album_type': 'album',
 'id': '0IuHVgAvbNDJnJepuSZ8Oz',
 'name': 'The Time Of Our Lives (International Version)',
 'release_date': '2009-01-01',
 'total_tracks': 8,
 'uri': 'spotify:album:0IuHVgAvbNDJnJepuSZ8Oz',
 'url': 'https://open.spotify.com/album/0IuHVgAvbNDJnJepuSZ8Oz',
 'image': 'https://i.scdn.co/image/871910879d96353fd909a24307625282cd5bbf69'}

In [58]:
df=pd.DataFrame([albumData][0])

In [116]:
df['artist id']=artistId
df['artist name']=df1.name

In [117]:
df.rename(columns = {'id':'album_id'}, inplace = True) 

In [118]:
df

,album_type,album id,image,name,release_date,total_tracks,uri,url,artist id,artist name
0,album,0IuHVgAvbNDJnJepuSZ8Oz,https://i.scdn.co/image/871910879d96353fd909a2...,The Time Of Our Lives (International Version),2009-01-01,8,spotify:album:0IuHVgAvbNDJnJepuSZ8Oz,https://open.spotify.com/album/0IuHVgAvbNDJnJe...,5YGY8feqx7naU7z4HrwZM6,Miley Cyrus
1,album,3wBabo4pmzsYjALMSKY7Iq,https://i.scdn.co/image/d431bed3736ab2699029b8...,Illuminate (Deluxe),2017-04-20,16,spotify:album:3wBabo4pmzsYjALMSKY7Iq,https://open.spotify.com/album/3wBabo4pmzsYjAL...,7n2wHs1TKAczGzO7Dd2rGr,Shawn Mendes
2,album,3KziODx7ieQTgYCVrS6nBv,https://i.scdn.co/image/94105e271865c28853bfb7...,Hollywood's Bleeding,2019-09-06,17,spotify:album:3KziODx7ieQTgYCVrS6nBv,https://open.spotify.com/album/3KziODx7ieQTgYC...,246dkjvS1zLTtiykXe5h60,Post Malone
3,single,0NVQ9k3wKmuK6T02lLMl6y,https://i.scdn.co/image/cc99138518f5c372f34713...,Breach,2018-11-08,4,spotify:album:0NVQ9k3wKmuK6T02lLMl6y,https://open.spotify.com/album/0NVQ9k3wKmuK6T0...,4GNC7GD6oZMSxPGyXy4MNB,Lewis Capaldi
4,album,3KziODx7ieQTgYCVrS6nBv,https://i.scdn.co/image/94105e271865c28853bfb7...,Hollywood's Bleeding,2019-09-06,17,spotify:album:3KziODx7ieQTgYCVrS6nBv,https://open.spotify.com/album/3KziODx7ieQTgYC...,246dkjvS1zLTtiykXe5h60,Post Malone
5,single,4mTzezatFBCJOr8s2QJrRY,https://i.scdn.co/image/68e8af4122e3502d997d6b...,It Will Feel,2019-09-27,1,spotify:album:4mTzezatFBCJOr8s2QJrRY,https://open.spotify.com/album/4mTzezatFBCJOr8...,1LnBqMojv6Dv6DoPWTDSZ7,Saco
6,album,7oc6i8xUILaWwQ5BJMZS3c,https://i.scdn.co/image/0a9862036a53c6c999b48b...,Tha Carter IV (Deluxe),2011-01-01,18,spotify:album:7oc6i8xUILaWwQ5BJMZS3c,https://open.spotify.com/album/7oc6i8xUILaWwQ5...,55Aa2cqylxrFIXC767Z865,Lil Wayne
7,single,71nefcDnnKgvaXbmo0EDYR,https://i.scdn.co/image/ab67616d0000b273079225...,Orange Juice (feat. Youkii),2019-09-27,1,spotify:album:71nefcDnnKgvaXbmo0EDYR,https://open.spotify.com/album/71nefcDnnKgvaXb...,7cyJzgiJTXR3hf62l5zLOr,Henri Purnell
8,single,2hBfao8GWZwHlUGDB8HVQO,https://i.scdn.co/image/ab67616d0000b273ad79d3...,Nothing Breaks Like a Heart (feat. Miley Cyrus),2018-11-30,1,spotify:album:2hBfao8GWZwHlUGDB8HVQO,https://open.spotify.com/album/2hBfao8GWZwHlUG...,3hv9jJF3adDNsBSIQDqcjp,Mark Ronson
9,single,4xP92MEGpK04DOLBGsyqYw,https://i.scdn.co/image/835c20ee8cef6b5a648c04...,Green Light,2019-10-04,1,spotify:album:4xP92MEGpK04DOLBGsyqYw,https://open.spotify.com/album/4xP92MEGpK04DOL...,4skzy1qEMyM3FqC3b6EjRU,Imad


In [119]:
#without the duplicates
len(set(artistId))

430

In [120]:
final_df = df.drop_duplicates('album id').sort_index()
print(len(final_df))

1196


In [121]:
final_df.to_csv(loc+"\\AlbumData.csv")

# A little detour to add total track of an artist from album table to artist table

In [134]:
artistTable=pd.read_csv(loc+"\\ArtistData.csv")
albumTable=pd.read_csv(loc+"\\AlbumData.csv")

In [135]:
albumTable.head()

,Unnamed: 0,album_type,album id,image,name,release_date,total_tracks,uri,url,artist id,artist name
0,0,album,0IuHVgAvbNDJnJepuSZ8Oz,https://i.scdn.co/image/871910879d96353fd909a2...,The Time Of Our Lives (International Version),2009-01-01,8,spotify:album:0IuHVgAvbNDJnJepuSZ8Oz,https://open.spotify.com/album/0IuHVgAvbNDJnJe...,5YGY8feqx7naU7z4HrwZM6,Miley Cyrus
1,1,album,3wBabo4pmzsYjALMSKY7Iq,https://i.scdn.co/image/d431bed3736ab2699029b8...,Illuminate (Deluxe),2017-04-20,16,spotify:album:3wBabo4pmzsYjALMSKY7Iq,https://open.spotify.com/album/3wBabo4pmzsYjAL...,7n2wHs1TKAczGzO7Dd2rGr,Shawn Mendes
2,2,album,3KziODx7ieQTgYCVrS6nBv,https://i.scdn.co/image/94105e271865c28853bfb7...,Hollywood's Bleeding,2019-09-06,17,spotify:album:3KziODx7ieQTgYCVrS6nBv,https://open.spotify.com/album/3KziODx7ieQTgYC...,246dkjvS1zLTtiykXe5h60,Post Malone
3,3,single,0NVQ9k3wKmuK6T02lLMl6y,https://i.scdn.co/image/cc99138518f5c372f34713...,Breach,2018-11-08,4,spotify:album:0NVQ9k3wKmuK6T02lLMl6y,https://open.spotify.com/album/0NVQ9k3wKmuK6T0...,4GNC7GD6oZMSxPGyXy4MNB,Lewis Capaldi
4,5,single,4mTzezatFBCJOr8s2QJrRY,https://i.scdn.co/image/68e8af4122e3502d997d6b...,It Will Feel,2019-09-27,1,spotify:album:4mTzezatFBCJOr8s2QJrRY,https://open.spotify.com/album/4mTzezatFBCJOr8...,1LnBqMojv6Dv6DoPWTDSZ7,Saco


In [124]:
#for every "artist id", I need to add the "total_tracks" from the albumTable such that the "ablum id" is unique while adding
totTracks=albumTable.total_tracks.tolist()
totTracks

[8,
 16,
 17,
 4,
 1,
 18,
 1,
 1,
 1,
 1,
 1,
 15,
 1,
 22,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 4,
 14,
 1,
 1,
 20,
 6,
 1,
 1,
 5,
 7,
 10,
 1,
 16,
 1,
 1,
 15,
 16,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 12,
 5,
 1,
 1,
 100,
 15,
 18,
 13,
 15,
 10,
 24,
 17,
 1,
 1,
 18,
 11,
 1,
 1,
 12,
 17,
 1,
 1,
 26,
 1,
 1,
 1,
 19,
 1,
 1,
 1,
 12,
 17,
 13,
 16,
 4,
 1,
 3,
 1,
 14,
 1,
 1,
 1,
 2,
 16,
 2,
 1,
 15,
 1,
 1,
 1,
 12,
 6,
 1,
 1,
 13,
 18,
 1,
 1,
 1,
 18,
 14,
 12,
 16,
 19,
 15,
 1,
 14,
 4,
 17,
 1,
 1,
 14,
 16,
 12,
 1,
 1,
 1,
 18,
 20,
 1,
 1,
 1,
 10,
 17,
 1,
 1,
 1,
 14,
 1,
 21,
 15,
 1,
 20,
 18,
 1,
 1,
 27,
 1,
 1,
 2,
 18,
 1,
 20,
 11,
 1,
 1,
 11,
 12,
 16,
 23,
 1,
 13,
 22,
 4,
 4,
 12,
 12,
 12,
 8,
 14,
 14,
 7,
 12,
 9,
 11,
 11,
 16,
 10,
 10,
 12,
 12,
 12,
 12,
 17,
 10,
 14,
 12,
 14,
 1,
 15,
 14,
 14,
 10,
 14,
 17,
 11,
 9,
 8,
 15,
 14,
 7,
 16,
 12,
 16,
 13,
 15,
 11,
 58,
 15,
 11,
 13,
 18,
 12,
 14,
 7,
 13,
 14,
 13,
 12

In [125]:
totTracks[0]=totTracks[0]+1
totTracks[0]

9

In [126]:
for i in range(1196):
    for j in range(i+1,1196):
        if(albumTable.artist_id[i]==albumTable.artist_id[j]):
            if(albumTable.album_id[i] != albumTable.album_id[j]):
                totTracks[i]=albumTable.total_tracks[i]+albumTable.total_tracks[j]
        else:
            continue

AttributeError: 'DataFrame' object has no attribute 'artist_id'

In [127]:
totTracks

[9,
 16,
 17,
 4,
 1,
 18,
 1,
 1,
 1,
 1,
 1,
 15,
 1,
 22,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 4,
 14,
 1,
 1,
 20,
 6,
 1,
 1,
 5,
 7,
 10,
 1,
 16,
 1,
 1,
 15,
 16,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 12,
 5,
 1,
 1,
 100,
 15,
 18,
 13,
 15,
 10,
 24,
 17,
 1,
 1,
 18,
 11,
 1,
 1,
 12,
 17,
 1,
 1,
 26,
 1,
 1,
 1,
 19,
 1,
 1,
 1,
 12,
 17,
 13,
 16,
 4,
 1,
 3,
 1,
 14,
 1,
 1,
 1,
 2,
 16,
 2,
 1,
 15,
 1,
 1,
 1,
 12,
 6,
 1,
 1,
 13,
 18,
 1,
 1,
 1,
 18,
 14,
 12,
 16,
 19,
 15,
 1,
 14,
 4,
 17,
 1,
 1,
 14,
 16,
 12,
 1,
 1,
 1,
 18,
 20,
 1,
 1,
 1,
 10,
 17,
 1,
 1,
 1,
 14,
 1,
 21,
 15,
 1,
 20,
 18,
 1,
 1,
 27,
 1,
 1,
 2,
 18,
 1,
 20,
 11,
 1,
 1,
 11,
 12,
 16,
 23,
 1,
 13,
 22,
 4,
 4,
 12,
 12,
 12,
 8,
 14,
 14,
 7,
 12,
 9,
 11,
 11,
 16,
 10,
 10,
 12,
 12,
 12,
 12,
 17,
 10,
 14,
 12,
 14,
 1,
 15,
 14,
 14,
 10,
 14,
 17,
 11,
 9,
 8,
 15,
 14,
 7,
 16,
 12,
 16,
 13,
 15,
 11,
 58,
 15,
 11,
 13,
 18,
 12,
 14,
 7,
 13,
 14,
 13,
 12

# Song Table

In [7]:
raw_playlist_data[0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [8]:
songData=[]
for i in range(len(raw_playlist_data)):
    d={}
    d['duration_ms']=raw_playlist_data[i]['duration_ms']
    d['explict']=raw_playlist_data[i]['explicit']
    d['external_urls']=raw_playlist_data[i]['external_urls']
    d['id']=raw_playlist_data[i]['id']
    d['name']=raw_playlist_data[i]['name']
    d['popularity']=raw_playlist_data[i]['popularity']
    d['uri']=raw_playlist_data[i]['uri']
    songData.append(d)

for i in range(len(songData)):
    songData[i]['url']=songData[i]['external_urls']['spotify']   
    del songData[i]['external_urls']

In [9]:
songData[0]

{'duration_ms': 202066,
 'explict': False,
 'id': '3E7dfMvvCLUddWissuqMwr',
 'name': 'Party In The U.S.A.',
 'popularity': 80,
 'uri': 'spotify:track:3E7dfMvvCLUddWissuqMwr',
 'url': 'https://open.spotify.com/track/3E7dfMvvCLUddWissuqMwr'}

In [10]:
sp.audio_features(songData[0]['id'])

[{'danceability': 0.652,
  'energy': 0.698,
  'key': 10,
  'loudness': -4.667,
  'mode': 0,
  'speechiness': 0.042,
  'acousticness': 0.00112,
  'instrumentalness': 0.000115,
  'liveness': 0.0886,
  'valence': 0.47,
  'tempo': 96.021,
  'type': 'audio_features',
  'id': '3E7dfMvvCLUddWissuqMwr',
  'uri': 'spotify:track:3E7dfMvvCLUddWissuqMwr',
  'track_href': 'https://api.spotify.com/v1/tracks/3E7dfMvvCLUddWissuqMwr',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3E7dfMvvCLUddWissuqMwr',
  'duration_ms': 202067,
  'time_signature': 4}]

## Audio Feature Extraction
to extract audio features of a song, all we need is the uri of that song. uri is available in raw_playlist_data

In [11]:
def audio_features(element):
    #Add new key-values to store audio features
    
    element['key'] = []
    element['mode'] = []
    element['time_signature'] = []
    element['acousticness'] = []
    element['danceability'] = []
    element['energy'] = []
    element['instrumentalness'] = []
    element['liveness'] = []
    element['loudness'] = []
    element['speechiness'] = []
    element['tempo'] = []
    element['valence'] = []
    #element['popularity'] = []
    #create a track counter
    
    features = sp.audio_features(element['uri'])
        
    #Append to relevant key-value
    element['key'].append(features[0]['key'])
    element['mode'].append(features[0]['mode'])
    element['time_signature'].append(features[0]['time_signature'])
    
    element['acousticness'].append(features[0]['acousticness'])
    element['danceability'].append(features[0]['danceability'])
    element['energy'].append(features[0]['energy'])
    element['instrumentalness'].append(features[0]['instrumentalness'])
    element['liveness'].append(features[0]['liveness'])
    element['loudness'].append(features[0]['loudness'])
    element['speechiness'].append(features[0]['speechiness'])
    element['tempo'].append(features[0]['tempo'])
    element['valence'].append(features[0]['valence'])
    #popularity is stored elsewhere
    #pop = sp.track(i)
    #element['popularity'].append(pop['popularity'])

In [12]:
for i in songData:
    audio_features(i)

In [13]:
for i in range(len(songData)):
    songData[i]['acousticness']=songData[i]['acousticness'][0]
    songData[i]['danceability']=songData[i]['danceability'][0]
    songData[i]['energy']=songData[i]['energy'][0]
    songData[i]['instrumentalness']=songData[i]['instrumentalness'][0]
    songData[i]['liveness']=songData[i]['liveness'][0]
    songData[i]['loudness']=songData[i]['loudness'][0]
    songData[i]['speechiness']=songData[i]['speechiness'][0]
    songData[i]['tempo']=songData[i]['tempo'][0]
    songData[i]['valence']=songData[i]['valence'][0]
    songData[i]['key']=songData[i]['key'][0]
    songData[i]['mode']=songData[i]['mode'][0]
    songData[i]['time_signature']=songData[i]['time_signature'][0]

In [14]:
songData[0]

{'duration_ms': 202066,
 'explict': False,
 'id': '3E7dfMvvCLUddWissuqMwr',
 'name': 'Party In The U.S.A.',
 'popularity': 80,
 'uri': 'spotify:track:3E7dfMvvCLUddWissuqMwr',
 'url': 'https://open.spotify.com/track/3E7dfMvvCLUddWissuqMwr',
 'key': 10,
 'mode': 0,
 'time_signature': 4,
 'acousticness': 0.00112,
 'danceability': 0.652,
 'energy': 0.698,
 'instrumentalness': 0.000115,
 'liveness': 0.0886,
 'loudness': -4.667,
 'speechiness': 0.042,
 'tempo': 96.021,
 'valence': 0.47}

In [15]:
df=pd.DataFrame([songData][0])

In [18]:
df.tail()

,acousticness,danceability,duration_ms,energy,explict,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,time_signature,uri,url,valence
1486,0.0964,0.766,224507,0.594,True,3Qi2o8uPKDByu6vFF1iq9d,0.000219,5,0.1040,-9.636,0,Influence,0,0.0511,104.957,4,spotify:track:3Qi2o8uPKDByu6vFF1iq9d,https://open.spotify.com/track/3Qi2o8uPKDByu6v...,0.350
1487,0.0150,0.569,222693,0.810,True,0bG61MsnxRxRHc007PxA2o,0.000000,2,0.0419,-4.593,1,Messin' Around,51,0.0473,157.965,4,spotify:track:0bG61MsnxRxRHc007PxA2o,https://open.spotify.com/track/0bG61MsnxRxRHc0...,0.784
1488,0.0539,0.354,179145,0.510,False,5f3FQBNEyJBquykRZwQITm,0.000000,10,0.2590,-8.849,0,Love Incredible (feat. Camila Cabello),0,0.1570,166.973,4,spotify:track:5f3FQBNEyJBquykRZwQITm,https://open.spotify.com/track/5f3FQBNEyJBquyk...,0.208
1489,0.1730,0.315,175517,0.563,False,5EbX5xL3O2xT8FxCpYzgUO,0.000242,9,0.0673,-9.671,1,"How Far I'll Go - From ""Moana""",72,0.3540,179.484,4,spotify:track:5EbX5xL3O2xT8FxCpYzgUO,https://open.spotify.com/track/5EbX5xL3O2xT8Fx...,0.147
1490,0.4740,0.329,278360,0.358,False,5zGPITTmaPqluxoPFYRsOY,0.003450,10,0.1100,-10.956,1,Love,7,0.0330,99.373,4,spotify:track:5zGPITTmaPqluxoPFYRsOY,https://open.spotify.com/track/5zGPITTmaPqluxo...,0.232


In [19]:
##just a small piece of code to remove duplicate elements from the table if any
print(len(df))
final_df = df.drop_duplicates('name').sort_index()
print(len(final_df))

1491
1132


In [20]:
final_df.tail()

,acousticness,danceability,duration_ms,energy,explict,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,time_signature,uri,url,valence
1483,0.4980,0.736,230813,0.795,True,6gpcs5eMhJwax4mIfKDYQk,0.000001,1,0.2540,-3.299,0,Slide (feat. Frank Ocean & Migos),75,0.0545,104.066,4,spotify:track:6gpcs5eMhJwax4mIfKDYQk,https://open.spotify.com/track/6gpcs5eMhJwax4m...,0.511
1485,0.0189,0.596,240426,0.838,False,6VZdDF6AgZHWiJfgs3T86v,0.000015,5,0.1080,-6.520,0,Water Under the Bridge,50,0.0704,94.982,4,spotify:track:6VZdDF6AgZHWiJfgs3T86v,https://open.spotify.com/track/6VZdDF6AgZHWiJf...,0.472
1486,0.0964,0.766,224507,0.594,True,3Qi2o8uPKDByu6vFF1iq9d,0.000219,5,0.1040,-9.636,0,Influence,0,0.0511,104.957,4,spotify:track:3Qi2o8uPKDByu6vFF1iq9d,https://open.spotify.com/track/3Qi2o8uPKDByu6v...,0.350
1487,0.0150,0.569,222693,0.810,True,0bG61MsnxRxRHc007PxA2o,0.000000,2,0.0419,-4.593,1,Messin' Around,51,0.0473,157.965,4,spotify:track:0bG61MsnxRxRHc007PxA2o,https://open.spotify.com/track/0bG61MsnxRxRHc0...,0.784
1488,0.0539,0.354,179145,0.510,False,5f3FQBNEyJBquykRZwQITm,0.000000,10,0.2590,-8.849,0,Love Incredible (feat. Camila Cabello),0,0.1570,166.973,4,spotify:track:5f3FQBNEyJBquykRZwQITm,https://open.spotify.com/track/5f3FQBNEyJBquyk...,0.208


In [23]:
final_df.to_csv(loc+"\\SongsData.csv")